this worksheets is part of the [mlvu machine learning course](https://mlvu.github.io)<br>
setting up your environment: https://bit.ly/3bzpn5C

For this worksheet, we'll look into the details of how a deep learning framework operates under the hood. **Tensorflow** and **Pytorch** are both good candidates, but things are a little easier to explain in Pytorch, so we'll use that. Step one, installation:

In [ ]:
!pip install torch

If the following cell executes without error, the installation was succesful.

In [ ]:
import torch

# Worksheet 5: Deep Learning with Pytorch

This worksheet assumes that you've watched the _Deep Learning 1_ and _Deep Learning 2_ lectures. If you don't know what backpropation means or what a computation graph is, you should probably give the lectures [a watch](https://mlvu.github.io/) first. If you want an even deeper understanding, try the lectures of the [DLVU course](https://dlvu.github.io), especially the second lecture.

The basic datastructure around which all of Pytorch is built is the _Tensor_. It works pretty much exactly like the tensors we've already seen in numpy.

To make a matrix filled with given values:


In [ ]:
torch.tensor([[1.0, 2.0], [3.0, 4.0]])

To make a matrix filled with (normally distributed) random numbers:


In [ ]:
torch.randn(3, 4)

For higher-dimensional matrices (aka tensors), just add more dimensions:

In [ ]:
torch.randn(2, 3, 2)

We can request the size of a tensor with the ```size()``` function. For the size of a particular dimension, just add the index of that dimension.

In [ ]:
x = torch.randn(3, 4, 2)

print(x.size())
print(x.size(0))

Just like in numpy, we can sum and multiply, and apply basic functions element-wise

In [ ]:
a, b = torch.randn(3, 4), torch.rand(3, 4)
c = torch.randn(4, 3)

# addition, multiplication, etc are all element-wise
summed = a + b
mult   = a * b
power  = a ** 2
sine   = torch.sin(a)

# _matrix_ multiplication is done through torch.mm
mmult = torch.mm(a, c)

# Note that the following lines would both give an error. Why?
# mult = a * c
# torch.mm(a, b)

What are the shapes of the tensors above (summed, mult, etc)? Print them to see if your intuition is correct.

Indexing and slicing works as it does in numpy.

In [ ]:
print(a)

print()
print('element at the first row and third column of a:', a[0, 2]) # note: zero-indexing
print('   first row of a:', a[0, :])
print('first column of a:', a[:, 0])

# You can also use basic slicing syntax; i:j refers to the range from i to j
# (more precisely, i is the first element included, j is the first element 
#  excluded)
print()
print('middle two elements of each row of a:\n', a[:, 1:3])

## tensor.view()


<div style="float:right; width: 250px">
<img src="https://upload.wikimedia.org/wikipedia/commons/4/4d/Row_and_column_major_order.svg" width="200px" title="row-major and column-major ordering" />
<small>By Cmglee - Own work, CC BY-SA 4.0, https://commons.wikimedia.org/w/index.php?curid=65107030</small>
</div>

One of the most important skills in programming in Pytorch is reshaping a tensor. It sounds simple, but there are some important subtleties to pay attention to. If you're not interested in the finer details of Pytorch, you can safely skip to the **backpropagation** section and save this part for later.

Computer memory is one big row of bits. However your numbers are arranged in your tensor, in memory, they have to be a  single sequence of of numbers. That means that if you have a matrix like
$$
\begin{pmatrix}
3 & 5 & 2\\
1 & 3 & 4\\
\end{pmatrix}
$$
it will be stored in memory as
$$
\begin{pmatrix}
3 & 5 & 2 & 1 & 3 & 4 \\
\end{pmatrix}\text{.}
$$
This is called **[row-major ordering](https://en.wikipedia.org/wiki/Row-_and_column-major_order)**: to get the memory layout of the matrix we scan first along the rows, and then along the columns.

Pytorch also stores the matrix dimensions, so it can compute that to get element $(2,1)$ in the matrix, it needs to access element 3 in the list.

For higher order tensors the principle is the same: the memory layout scans first over the rightmost dimension. So, if we have a tensor $A$ with size ```(2, 2, 3)```, the elements are stored in the order:
$$
A_{111}, A_{112}, A_{113}, A_{121}, A_{122}, A_{123}, A_{211}, A_{212}, \ldots
$$
Note that at every step the rightmost index increments first. When it gets to its maximum, the one to the left of it increments, and so on.

### reshaping

We can take the data of one matrix in memory, and create a second matrix from it with another shape. This is done by the ```view()``` function, it takes a tensor and gives you a new _view_ on the same data, assuming a different shape:


In [ ]:
matrix = torch.tensor([[3,5,2,6],[1, 3, 4,0],[-1,-3, -4,-0]])

print(matrix) 
print(matrix.view(4, 3))
print(matrix.view(2, 6))
print(matrix.view(size=(12,)))


You can use ```-1``` for one of the arguments. Pytorch will work out what the size of that dimension is from the rest of the values.

In [ ]:
print(matrix.view(-1, 6))
print(matrix.view(-1,))
# print(matrix.view(-1, 3, -1)) # this doesn't work


Note the difference between matrix reshaping and the matrix transpose (done with the ```.t()``` function):


In [ ]:
matrix = torch.tensor([[3,5,2],[1, 3, 4]])

print(matrix.view(3, 2))
print(matrix.t())

Transposing can _also_ be done cheaply. Pytorch creates a wrapper around the old matrix that remaps the dimensions. Requesting element $(i, j)$ in the new matrix is remapped to a request for $(j, i)$ in the old matrix. However, in this case, the new matrix is not **contiguous** anymore: the sequence in memory is no longer in row-major order for the new shape. Pytorch can do some things with non-contiguous matrics, but not all. Calling some functions on a non-contiguous matrix will cause an error. 

For instance, if you try to reshape a non-contiguous matrix with ```view()```, pytorch will complain. The solution is to copy and rearrange the original memory to a new sequence in memory that is contiguous for this matrix shape.



In [ ]:
# matrix.t().view(2, 3)              # this fails
matrix.t().contiguous().view(2, 3)   # this works, but copies the matrix

When you use ```reshape()``` instead of view, the matrix is made contiguous if that is necessary, but otherwise the same data is used. This is nice if you don't care much about memory use. If you want to be sure that you're not accidentally copying a large matrix, you should use ```view()```, and check for errors.

In [ ]:
matrix.t().reshape(2, 3) # this works, but copies the matrix without warning you

### Example: reshaping back and forth

Why is this important? Let's look at an example. Let's say you have a large dataset of N images, each with 3 color channels (RGB values) and WxH pixels. You can represent this in a big 4 tensor:

In [ ]:
N, C, H, W = 100, 3, 32, 32
images = torch.rand(N, H, W, C) # just random values for the example
print(images.size())

# -- The normal layout for images in pytorch if channels-first, but 
#    we'll put the channels at the endhere to simplify the example

Now let's say that you want to normalize the data _by color_. That is, we want to rescale the data so that the maximum value in each color channel is 1. We can find the maximum value in a particular dimension, with pytorch (using ```tensor.max(dim=x)```), but to do this over three dimensions requires taking the max three times. 

An alternative approach is to _reshape_ our data into a matrix, then normalize, and reshape back to the original data.

The main thing to remember in this sort of scenario is that we can safely reshape dimensions **that are next to each other**. Here, we want to combine the N, H and W dimensions, and keep the C dimension separate. That means that if we call

In [ ]:
images = images.view(N*H*W, C)

```images``` is now a _matrix_, with the N, H and W dimensions flattened (in row-major order) along the  vertical dimension. We can now easily normalize this matrix vertically:

In [ ]:
images = images / images.max(dim=0)[0]

# -- Note that even though the numerator has size (102400, 3) and 
#    the denominator has size(3), we can still do element-wise division,
#    because pytorch supports broadcasting, the same as numpy. It's 
#    worth knowing _exactly_ how broadcasting works, or the results
#    can be surprising. Check the numpy worksheet for more information.

Now that all the colors are normalized, we can simply reshape our data back into images.

In [ ]:
images = images.view(N, H, W, C)

This sort of thing can be very useful, but you have to be careful to keep in mind what the layout of the data is in memory. Pytorch happily let you do

In [ ]:
images = images.view(N*H, W*C)
images = images.view(H, N, C, W)

but the result is is basically a completely scrambled dataset. 

In short, make sure you understand the difference between _transposing_ (swapping dimensions) and reshaping (changing the dimensions, but keeping the data the same).

## Backpropagation

Let's look at how Pytorch implements _backpropagation_. All we need to do is create some tensors, tell Pytorch that we want to compute gradients for them, and then do some operations on them that result in a single scalar.

In [ ]:
a, b = torch.randn(3, 4), torch.rand(3, 4)   # create some tensors ...
a.requires_grad = True   # ... tell pytorch that we want gradients on a ...

out = ((a + b) ** 2).sum()   # ... and perform a computation resulting in a scalar value.
print(out)

Pytorch has not just computed the result (```out```), it's also included a pointer to a ```SumBackward``` object representing the computation (summing) that created ```out```. This object links back to other objects, all the way down to the start of the computation.

In [ ]:
print(out.grad_fn) # sum
print(out.grad_fn.next_functions[0][0]) # raise to a power
print(out.grad_fn.next_functions[0][0].next_functions[0][0]) # addition

# Note: these are not attributes you would normally use. We just call them here to show that 
# Pytorch is remembering everything we do.


We've asked Pytorch to ensure we can compute a gradient on ```a``` and done some basic computation. The computation has resulted in a single number (```out```), so we can now compute the gradient of ```a``` over that output. Remember, backpropagation only works efficiently if the output of the computation graph is a single scalar, usually your loss.

In [ ]:
print(a.grad)     # this is the gradient on a. Note it's currently empty

out.backward()    # ask Pytorch to perform backprop

print(a.grad)     # now a has a gradient

Note that the gradient of ```a``` has the same shape as ```a```.

## Learning

Pytorch has many utilities to help you quickly build elaborate networks, but it's instructive to first see how you would use just these tools to build a simple model. As an example, we will build a simple linear regression model.

First, let's generate some simple random data.

In [ ]:
x = torch.randn(1000, 32)                      # 1000 instances, with 32 features
wt, bt = torch.randn(32, 1), torch.randn(1)    # function to compute the true labels
t = torch.mm(x, wt) + bt                       # the true target labels

Next up, we define the parameters of our model (we'll initialize them randomly).

In [ ]:
w = torch.randn(32, 1, requires_grad=True) 
b = torch.randn(1, requires_grad=True)

Note that any method that creates tensors (like ```torch.randn()```) can be told that it should make them require a gradient).

Here's what one computation of the model output over the whole data looks like. We'll print the shapes of the tensors to see what's going on. **Before you run this cell, see if you can work out what the sizes will be.**

In [ ]:
print('          data size:', x.size())

# model output
y = torch.mm(x, w) + b

print('        output size:', y.size())

print()
print('first 3 predictions:', y[:3, 0])
print('       ground truth:', t[:3, 0]) # note that these will be completely different, because
                                        # we haven't started training yet

# residuals
r = t - y
print()
print('     residuals size:', r.size())
    
# mean-squared-error loss 
loss = (r ** 2).mean()
print()
print('               loss:', loss.item())
# -- if you have a tensor with a single number, .item() will turn it into a normal float for you.

    

We can now apply backpropagation, and see that we get a gradient over our two parameters ```w``` and ```b```. Before you run the cell, what will the sizes of the gradient tensors be?

In [ ]:
loss.backward()

print('gradient on w:', w.grad)
print('gradient on b:', b.grad)

# NB: if you run the cell twice, pytorch will complain. After each backward, pytorch expects a new forward.

We are now ready to build a training loop. We'll use basic gradient descent without minibatches, computing the loss over the whole data every iteration. 

In [ ]:
# hyperparameters
iterations = 21
learning_rate= 0.5

# regenerate the data and model
x = torch.randn(1000, 32)                      # 1000 instances, with 32 features
wt, bt = torch.randn(32, 1), torch.randn(1)    # parameters of the true model
t = torch.mm(x, wt) + bt 

w = torch.randn(32, 1, requires_grad=True) 
b = torch.randn(1, requires_grad=True)

for i in range(iterations):

    # forward pass
    y = torch.mm(x, w) + b

    # mean-squared-error loss 
    r = t - y
    loss = (r ** 2).mean()

    # backpropagation
    loss.backward()
        
    # print the loss
    print(f'iteration {i: 4}: loss {loss:.4}')
    
    # gradient descent
    w.data = w.data - learning_rate * w.grad.data
    b.data = b.data - learning_rate * b.grad.data
    # -- Note that we don't want the autodiff engine to compute gradients over this part.
    #   by operrating on w.data, we are only changing the values of the tensor not 
    #   remembering a computation graph.

    # delete the gradients
    w.grad.data.zero_()
    b.grad.data.zero_()
    # -- if we don't do this, the gradients are remembered, and any new gradients are added
    #    to the old.   

# show the true model, and the learned model
print()
print('true model: ', wt.data[:4].t(), bt.data)
print('learned model:', w.data[:4].t(), b.data)
    

## torch.optim

While you can build everything yourself using these low-level objects, many people have done so before and packaged the results in reusable libraries. 

We'll first look at ```torch.optim```, which contains a number of _optimizers_. Using these, we don't have to implement the gradient descent step ourselves. This may not seem like a big part of our code, but it can get more complicated when we want to try variations on gradient descent like Adam. To illustrate, let's use the Adam optimizer in our linear regression example.

In [ ]:
from torch.optim import Adam

# hyperparameters
iterations = 101
learning_rate= 2.0

# regenerate the data and model
x = torch.randn(1000, 32)                      # 1000 instances, with 32 features
wt, bt = torch.randn(32, 1), torch.randn(1)    # parameters of the true model
t = torch.mm(x, wt) + bt 

w = torch.randn(32, 1, requires_grad=True) 
b = torch.randn(1, requires_grad=True)

# Create the optimizer. It needs to know two things:
# - the learning rate
# - which parameters its responsible for
opt = Adam(lr=learning_rate, params=[w, b])

for i in range(iterations):

    # forward/backward, same as before
    y = torch.mm(x, w) + b
    r = t - y
    loss = (r ** 2).mean()
    
    loss.backward() 
    # -- Note that the optimizer _doesn't_ compute the gradients. We still
    #    do that ourselves. The optimizer takes the gradients, and uses them
    #    to adapt the parameters. 
        
    # print the loss
    if i % 20 == 0:
        print(f'iteration {i: 4}: loss {loss:.4}')
    
    # Perform the gradient descent step
    opt.step() 
    
    # The optimizer can zero the gradients for us 
    # (but we still have to tell it to do so)
    opt.zero_grad()

You may have noticed that Adam (which is supposed to be better than gradient descent in many ways), actually takes longer to converge. That's because this is a very simple problem. Adam's strength shows when you train very large networks with many weights all doing different things.

## torch.nn

The package ```torch.nn``` contains utilities for building the kind of large, complex neural networks we've seen in the lectures. It is built around <em>modules</em>: modules are classes that combine a set of weights with a ```forward()``` function that uses these weights to compute a forward pass.

The simplest module is probably ```torch.nn.Linear```. It implements a simple linear operation with a weight matrix and a bias vector (this is like the ```Dense``` layer in Keras).

In [ ]:
from torch import nn

lin = nn.Linear(2, 2) # a linear function from a 2-vector to a 2-vector

print('weight matrix:', lin.weight)
print()
print('bias vector:', lin.bias)

You can see that Pytorch knows that ```lin.weight``` and ```lin.bias``` are _parameters_. This is because they are ```nn.Parameter``` objects, a lightweight wrapper around the torch tensor, which signals that this tensor is meant to be treated like a model parameter. It has ```requires_grad=True``` by default, and there are helper functions to collect all parameters of a complex model.

We can apply the linear transformation by calling ```lin``` just like a function.

In [ ]:
x = torch.tensor([1.0, 2.0])
lin(x)

Note that the resulting tensor has a ```grad_fn``` attribute, so we can tell that the computation graph is being remembered.

To implement a module of our own, we create a subclass of the ```nn.Module``` class. All we need to implement is the constructor and the ```forward``` function. Here is a module for a simple two-layer MLP with a ReLU activation on the hidden layer.

To illustrate how to define and apply parameters, we will also add a multiplier to the output (a single learnable value). 

In [ ]:
import torch.nn.functional as F # some helpful utility functions

class MLP(nn.Module):
    
    def __init__(self, in_size = 16, hidden_size=32, out_size=1):
        """
        This is the _constructor_ the function that creates an instance of the MLP class.
        
        The argument 'self' is a standard argument in python object-oriented programming. It
        refers to the current instance that we're creating. The other arguments are parameters
        of the MLP.
        """
        super().__init__()
        
        # everything that has parameters should be created in the contructor
        self.layer1 = nn.Linear(in_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, out_size)
        
        # the layers have most of the parameters, but we will also add one of our own
        self.mult = nn.Parameter(torch.tensor([1.0]))
        # -- we create a tensor with the initial value, and wrap it in an nn.Parameter
        #    objects. Because it's an nn.Parameter, pytorch will take care of the rest.
        
    def forward(self, x):
        """
        This is the function that gets executed when we call the module like a  function.

        - The argument 'self' again refers to the current object.
        - The argument 'x' is the input to the function (multiple arguments, named aguments 
          and even no arguments are possible)
        """        

        h = self.layer1(x)  # apply the first layer
        h = F.relu(h)       # apply a ReLU nonlinearity
        o = self.layer2(h)  # apply the second layer    
        
        o = o * self.mult        # apply the multiplier

        return o

We can now create an MLP instance, and feed it some data. Before you run the cell, can you predict the size of the output?

In [ ]:
mlp = MLP()            # create an MLP with the standard dimensions

x = torch.randn(3, 16) # three instances, with 16 features

mlp(x)                 # pass the data through the MLP.

Because we've subclassed ```nn.Module```, we get a lot of functionality for free. For instance, ```mlp``` has a function that lets us loop over all its parameters and the parameters of its modules (and the parameters of their modules and so on).

In [ ]:
for param in mlp.parameters():
    print(param.size())

In order, these are the multiplier, the weights matrix of the first layer, the bias of the first layer, the weight matrix of the second layer and the bias of the second layer.

This is helpful when we need to let the optimizer know what the parameters of our model are. 

Here's an example of how to put everything together and train our MLP on some generated data

In [ ]:
# hyperparameters
iterations = 1000
learning_rate= 0.01

# regenerate the data and model
x = torch.randn(1000, 32)                          # 1000 instances, with 2 features
t = torch.sqrt(x.pow(2).sum(dim=1, keepdim=True))  # we'll use the vector norm as a target function

model = MLP(32, 64, 1)

opt = Adam(lr=learning_rate, params=model.parameters())
# -- Note that we just point the optimizer to the parameters generator

for i in range(iterations):

    y = model(x)
    loss = F.mse_loss(y, t) 
    # -- We'll switch to the pytorch implementation of the MSE. 
    
    if i % 50 == 0:
        print(f'iteration {i: 4}: loss {loss:.4}')
    
    loss.backward()
    
    opt.step()
    opt.zero_grad()

## Piece de resistance: the variational autoencoder

In the Keras worksheet, we built an autoencoder. Now, let's build a _variational_ autoencoder (VAE). This possible in Keras as well, but several aspects of the VAE make it a bit awkward: we have one loss at the end and one loss halfway down the network. Also we have to implement a sampling step in the middle. 

In pytorch all this becomes a bit easier, because the forward pass of our model looks so much more like regular code.

**Note that this is only a tutorial on how to _build_ a VAE. We'll expect you to know how a VAE works already. If you don't, please review the second Deep Learning lecture.**

We'll start by importing the MNIST data. First we have to install the ```torchvision``` package.

In [ ]:
!pip install torchvision

Then, we load the data. In pytorch, data is usually packaged in a dataloader, which can efificiently serve you batches of data. To simplify things, we'll only load the training data. We will ask for batches of 32 images.

In [ ]:
import torchvision
from torchvision import transforms

train = torchvision.datasets.MNIST(root='./mnist', train=True, download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True, num_workers=2)

# We use the trainloader by looping over it. In this case, each batch is a pair of an image tensor, and a 
# label tensor. We'll ignore the labels.
for images, labels in trainloader:
    print(images.size(), labels.size())
    break


Next up, we'll define the model. As we did in the Keras worksheet, we'll define the encoder and decoder separately

In [ ]:
# Pytorch doesn't give us a Reshape module. We'll add that ourselves so we 
# can define the encoder and decoder as sequences of operations
class Reshape(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape

    def forward(self, input):
        return input.view( (input.size(0),) + self.shape) # keep the batch dimensions, reshape the rest

# - channel sizes
a, b, c = 16, 32, 128
latent_size = 2

encoder = nn.Sequential(
    nn.Conv2d(1, a, (3, 3), padding=1), nn.ReLU(),
    nn.MaxPool2d((2, 2)),
    nn.Conv2d(a, b, (3, 3), padding=1), nn.ReLU(),
    nn.Conv2d(b, b, (3, 3), padding=1), nn.ReLU(),    
    nn.MaxPool2d((2, 2)),
    nn.Conv2d(b, c, (3, 3), padding=1), nn.ReLU(),
    nn.Conv2d(c, c, (3, 3), padding=1), nn.ReLU(),
    nn.MaxPool2d((2, 2)),
    nn.Flatten(),
    nn.Linear(3 * 3 * c, 2 * latent_size)
)

decoder = nn.Sequential(
    nn.Linear(latent_size, c * 3 * 3), nn.ReLU(),
    Reshape((c, 3, 3)),
    nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
    nn.ConvTranspose2d(c, b, (3, 3), padding=1), nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
    nn.ConvTranspose2d(b, a, (3, 3), padding=0), nn.ReLU(), # note the padding
    nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
    nn.ConvTranspose2d(a, 1, (3, 3), padding=1), nn.Sigmoid()
)



We are cheating a little with the output activation. To truly follow the derivation of the VAE, this should define a distribution in the data space (which are continuous numbers _between_ 0 and 1). In this case, we use a sigmoid activation with a binary cross-entropy, which would be a distribution for binary data (either 0 or 1). 

A more correct VAE would use, for instance, a Gaussian output, which boils down to an MSE loss (but doesn't work well for this task), or add some terms to make the BCE loss work theoretically. For now we'll ignore this and stick with a plain BCE loss.

More importantly, note that the output of the encoder is _twice_ the size of the latent space, while the input to the decoder is the size of the latent space. This is because the decoder gives us a mean _and a variance_ on the latent space, from which we'll _sample_ the input to the decoder.

We'll also need to compute the KL loss term from this mean and variance. We'll introduce some utility functions for both operations.

In [ ]:
def kl_loss(zmean, zsig):
    b, l = zmean.size()

    kl = 0.5 * torch.sum(zsig.exp() - zsig + zmean.pow(2) - 1, dim=1)
    # -- The KL divergence between a given normal distribution and a standard normal distribution
    #    can be rewritten this way. It's a good exercise to work this out.

    assert kl.size() == (b,)
    # -- At this point we want the loss to be a single value of each instance in the batch.
    #    Asserts like this are a good way to document what you know about the shape of the 
    #    tensors you deal with.

    return kl

def sample(zmean, zsig):
    b, l = zmean.size()

    # sample epsilon from a standard normal distribution
    eps = torch.randn(b, l)

    # transform eps to a sample from the given distribution
    return zmean + eps * (zsig * 0.5).exp()

If you want to try to work out why the KL term looks like that, you should [start here](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence#Multivariate_normal_distributions) and try to rewrite to pytorch code.

Now that we have all the steps in our model, we can build the training loop.

Since this might take a while, we install and import [```tqdm```](https://tqdm.github.io/) to give us some nice progress bars.

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

parameters = list(encoder.parameters()) + list(decoder.parameters()) # -- retrieve all paremeters of both models
opt = Adam(lr=0.0003, params=parameters)

for epoch in range(5):
    for images, _ in tqdm(trainloader): # if tqdm gives you trouble just remove it
        b, c, h, w = images.size()
                
        # forward pass
        z = encoder(images) 
        
        # - split z into mean and sigma
        zmean, zsig = z[:, :latent_size], z[:, latent_size:]
        kl = kl_loss(zmean, zsig)
        
        zsample = sample(zmean, zsig)
        
        o = decoder(zsample)
        rec = F.binary_cross_entropy(o, images, reduction='none')
        rec = rec.view(b, c*h*w).sum(dim=1)
        # -- Reconstruction loss. We ask pytorch not to sum the loss, and sum over the 
        #    channels and pixels ourselves. This gives us a loss per instance that we 
        #    can add to the kl loss
        
        loss = (rec + kl).mean() # sum the losses and take the mean
        loss.backward()
        
        opt.step()
        opt.zero_grad()
    
    print(f'epoch {epoch}: {loss.item()}')
        

Training takes about 5 minutes per epoch on my laptop (note that the autoencoder is a bit bigger than the one in the Keras worksheet). I recommend letting it run for at least 3 epochs.

To visualize what our autoencoder did, we'll plot some of the data by their latent space coordinates. That is, for each image, we compute the latent space coordinates, and plot the original image at that point (this takes a while):

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import math
import numpy as np

# gather up first 200 batches into one big tensor
numbatches = 200 # -- change 200 to a lower number to speed things up
images, labels = [], []
for i, (ims, lbs) in enumerate(trainloader):
    images.append(ims)
    labels.append(lbs)
    
    if i > numbatches:
        break
    
images, labels = torch.cat(images, dim=0), torch.cat(labels, dim=0)

n, c, h, w = images.size()

z = encoder(images)
latents = z[:, :2].data

mn, mx = latents.min(), latents.max()
size = 1.0 * (mx-mn)/math.sqrt(n)
# Change 0.75 to any value between ~ 0.5 and 1.5 to make the digits smaller or bigger

fig = plt.figure(figsize=(16,16))

# colormap for the images
norm = mpl.colors.Normalize(vmin=0,vmax=9)
cmap = mpl.cm.get_cmap('tab10')

for i in range(n):
    
    x, y = latents[i, 0:2]
    l = labels[i]
    
    im = images[i, :]
    alpha_im = im.permute(1, 2, 0).numpy()
    color = cmap(norm(l))
    color_im = np.asarray(color)[None, None, :3]
    color_im = np.broadcast_to(color_im, (h, w, 3))
    # -- To make the digits transparent we make them solid color images and use the 
    #    actual data as an alpha channel.
    #    color_im: 3-channel color image, with solid color corresponding to class
    #    alpha_im: 1-channel grayscale image corrsponding to input data

    im = np.concatenate([color_im, alpha_im], axis=2)
    plt.imshow(im, extent=(x, x + size, y, y + size))

    plt.xlim(mn, mx)
    plt.ylim(mn, mx)
    

We see that even with just 2 latent dimensions, the VAE manages to separate out the digits quite well (remember, the model doesn't see the labels, they are only used to color the plot). Even within a digit's, the variations (like angle, and stroke thickness) are being grouped together.

You can try to make the separation clearer by training for more epochs, by adding more channels or layers to the encoder or decoder, by tuning the learning rate more precisely.

## Conclusion

That's it, you've finished the last worksheet. Pytorch is a complicated system, so don't feel too bad if you don't understand all the details. Play around with the code, and follow some of the links below, and you'll soon deepen your understanding.

### Further reading

* Pytorch 60 minute blitz: https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html
* Learning Pytorch with examples: https://pytorch.org/tutorials/beginner/pytorch_with_examples.html
* Visualizing Models, Data, and Training with TensorBoard: https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html